# workflow.ipynb

In [1]:
import os
# import glob
# import shutil
import pandas as pd
import time
import papermill

In [2]:
DATADIR = '/home/aubrey/Desktop/test'
CATALOG_PATH = '/home/aubrey/Desktop/tinian_sticky_trap_images/catalog.csv'

In [3]:
start = time.time()

In [4]:
def halt_here():
    raise Exception('halt_here executed.')
    
# stop_here()

In [5]:
print('Downloading data from GitHub repo')
x = papermill.execute_notebook(
    input_path = 'get_dataset.ipynb',
    output_path = 'temp.ipynb',
    parameters={
        'REPO': 'https://github.com/aubreymoore/Tinian-sticky-traps-2022-02-26',
        'DATADIR': '/home/aubrey/Desktop/test'})

Executing:   0%|          | 0/10 [00:00<?, ?cell/s]

In [6]:
print('Cropping original images.')
df_imagelist = pd.read_csv(f'{DATADIR}/imagelist.csv')
for i, r in df_imagelist.iterrows():
    sn = os.path.basename(r.filename).replace('.png', '')
    ORIGINAL_IMAGE_PATH = f'{DATADIR}/{sn}/original.png'
    CROPPED_IMAGE_PATH =  f'{DATADIR}/{sn}/cropped.png'
    print(f'   Processing{DATADIR}/{sn}')
    x = papermill.execute_notebook(
        input_path = '/home/aubrey/Desktop/sticky_trap_imaging/code/crop_square_centered_on_qrcode.ipynb',
        output_path = 'temp.ipynb',
        parameters={
            'ORIGINAL_IMAGE_PATH': ORIGINAL_IMAGE_PATH, 
            'CROPPED_IMAGE_PATH': CROPPED_IMAGE_PATH})

Cropping original images.
   Processing/home/aubrey/Desktop/test/22


Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

   Processing/home/aubrey/Desktop/test/24


Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

   Processing/home/aubrey/Desktop/test/16


...

Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

   Processing/home/aubrey/Desktop/test/10


Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

   Processing/home/aubrey/Desktop/test/21


Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

In [7]:
print('Extracting insect images.')
for i, r in df_imagelist.iterrows():
    sn = os.path.basename(r.filename).replace('.png', '')
    IMAGEPATH = f'{DATADIR}/{sn}/cropped.png'
    OUTPUTPATH = f'{DATADIR}/{sn}'
    print(f'{sn = }')
    x = papermill.execute_notebook(
        input_path = 'extract_insect_images.ipynb',
        output_path = 'temp.ipynb',
        parameters={
            'IMAGEPATH': f'{DATADIR}/{sn}/cropped.png', 
            'OUTPUTPATH': f'{DATADIR}/{sn}', 
            'CANNY': 45})

Extracting insect images.
sn = '22'


Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

sn = '24'


Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

sn = '16'


...

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

sn = '10'


Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

sn = '21'


Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

In [8]:
print('Building catalog.')
x = papermill.execute_notebook(
    input_path = '/home/aubrey/Desktop/sticky_trap_imaging/code/build_catalog.ipynb',
    output_path='temp.ipynb',
    parameters={
        'FILE_LIST_GLOB': f'{DATADIR}/*/*-[0-9][0-9][0-9].png',
        'CATALOG_PATH': f'{DATADIR}/catalog.csv',
        'PIXELS_PER_MM': 36.375})

Building catalog.


Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

In [9]:
# Run clustimage for each size_class. Cluster will be added to catalog.
# If there is only one image in the size_class, cluster analysis is skipped and the image be in cluster -1 within the size_class

print('Running clustimage.ipynb.')
pd.set_option('max_colwidth', 0)
df = pd.read_csv(CATALOG_PATH)
size_classes = df['size_class'].unique().tolist()

for size_class in size_classes:
    size_class = size_class.replace(' ', '-')
    print(f'{size_class = }')
    x = papermill.execute_notebook(
        input_path = 'clustimage.ipynb',
        output_path = 'temp.ipynb',
        parameters={
            'CATALOG_PATH': f'{DATADIR}/catalog.csv', 
            'SIZE_CLASS': size_class}    )

Running clustimage.ipynb.
size_class = '0.1-to-0.2-mm'


Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

size_class = '0.3-to-0.4-mm'


Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

size_class = '0.4-to-0.5-mm'


...

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

size_class = '9-to-10-mm'


Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

size_class = '6-to-7-mm'


Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

In [10]:
print('Running create_cluster_folders.ipynb.')
x = papermill.execute_notebook(
    input_path = '/home/aubrey/Desktop/sticky_trap_imaging/code/create_cluster_folders.ipynb',
    output_path = 'temp.ipynb',
    parameters={
        'FILE_LIST_GLOB': f'{DATADIR}/*/*-[0-9][0-9][0-9].png',
        'CATALOG_PATH': f'{DATADIR}/catalog.csv',
        'PIXELS_PER_MM': 36.375})

Running create_cluster_folders.ipynb.


Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

In [12]:
end = time.time()
seconds = end - start
minutes = seconds / 60
hours = minutes / 60
print(f'FINISHED: Total run time was {seconds:.3g} seconds = {minutes:.3g} minutes = {hours:.3g} hours')

FINISHED: Total run time was 346 seconds = 5.77 minutes = 0.0961 hours
